In [1]:
#Import packages
import pandas as pd
from datetime import datetime

In [2]:
#Read data
df_raw = pd.read_excel('../data/raw/MadreDeDios_MercuryMasterData.xlsx')
df_raw.shape

(4627, 12)

In [3]:
#Explore
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4627 entries, 0 to 4626
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Author                       4627 non-null   object 
 1   Date data was collected      4627 non-null   object 
 2   Sample type                  4627 non-null   object 
 3   Species                      4223 non-null   object 
 4   Sample size                  4627 non-null   int64  
 5   Longitude (X)                4626 non-null   float64
 6   Latitude (Y)                 4626 non-null   float64
 7   Mercury concentration (ppm)  4627 non-null   float64
 8   Margin of error              766 non-null    float64
 9   Mercury type                 4627 non-null   object 
 10  Age (human only)             3191 non-null   float64
 11  Sex (human only)             3191 non-null   object 
dtypes: float64(5), int64(1), object(6)
memory usage: 433.9+ KB


### Task 1 - Remove records missing location data
Note in the result above, one record is missing latitude and longitude data. Here we'll remove that row.

In [4]:
#Remove the record with missing lat/lng data
df_raw.dropna(subset=['Longitude (X)','Latitude (Y)'],inplace=True)
df_raw.shape

(4626, 12)

### Task 2 - Fix the dates
The dates come in three formats. First, some records have proper date time formats (e.g. `2017-08-15 15:30:00`). Others, however, just have years (e.g. `2009`) or spans of years (e.g. `2001-2003`). 

To fix this, we'll construct three new columns:
* `DateCollected` will include the full date and time of collection for those records where it is provided and others being set to null values.  

* `StartYear` will include the "first" year collected. For records with a full datetime or just a single year, this will be the year of that record. For records with a range, this will be the first year of that range.  

* `EndYearCollected` will repeat the year collected if the record is has a full date time or just single year. Otherwise it will list the last year for the range specified.  

In [6]:
#%% Create conversion functions
def convert_date(the_date):
    if type(the_date) == datetime:
        out_date = the_date
        start_year = out_date.year
        end_year = out_date.year
    elif type(the_date) == type(1):
        out_date = None 
        start_year = the_date#datetime.strptime(str(the_date), '%Y')
        end_year   = the_date#datetime.strptime(str(the_date), '%Y')
    else: 
        out_date = None
        first_year,last_year = the_date.split("-")
        start_year = first_year #datetime.strptime(str(first_year), '%Y')
        end_year   = last_year  #datetime.strptime(str(last_year), '%Y')
    return pd.Series([out_date, int(start_year), int(end_year)])

#Apply function, adding the new fields
df_raw[['DateCollected','StartYear','EndYear']] = df_raw['Date data was collected'].apply(convert_date)

#Fix the fact that integers are upcasted as floats
df_raw.StartYear = df_raw.StartYear.astype('int')
df_raw.EndYear = df_raw.EndYear.astype('int')

#Reveal the timespan of records
print(f"Records span from {df_raw['StartYear'].min()} to {df_raw['EndYear'].max()}")

Records span from 1990 to 2017


In [12]:
#Save full table
df_raw.to_csv('../data/processed/MadreDeDios_MercuryMasterData.csv',index=False)

In [11]:
#Subset records with full timestamps
df_timestamps = df_raw[df_raw['DateCollected'].isna() == False]
df_timestamps

,Author,Date data was collected,Sample type,Species,Sample size,Longitude (X),Latitude (Y),Mercury concentration (ppm),Margin of error,Mercury type,Age (human only),Sex (human only),DateCollected,StartYear,EndYear
13,Martinez 2018,2017-01-26 08:00:00,Fish,Prochilodus nigricans,1,-69.520278,-13.020000,0.10,0.06,Total Mercury,NaN,NaN,2017-01-26 08:00:00,2017,2017
14,Martinez 2018,2017-01-26 08:00:00,Fish,Prochilodus nigricans,1,-69.520278,-13.020000,0.10,0.06,Total Mercury,NaN,NaN,2017-01-26 08:00:00,2017,2017
15,Martinez 2018,2017-01-26 08:00:00,Fish,Prochilodus nigricans,1,-69.520278,-13.020000,0.10,0.06,Total Mercury,NaN,NaN,2017-01-26 08:00:00,2017,2017
16,Martinez 2018,2017-01-26 08:00:00,Fish,Prochilodus nigricans,1,-69.520278,-13.020000,0.10,0.06,Total Mercury,NaN,NaN,2017-01-26 08:00:00,2017,2017
17,Martinez 2018,2017-01-26 08:00:00,Fish,Prochilodus nigricans,1,-69.520278,-13.020000,0.10,0.06,Total Mercury,NaN,NaN,2017-01-26 08:00:00,2017,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,Martinez 2018,2017-08-18 13:00:00,Fish,Prochilodus nigricans,1,-68.736111,-12.693333,0.10,0.06,Total Mercury,NaN,NaN,2017-08-18 13:00:00,2017,2017
317,Martinez 2018,2017-08-18 13:00:00,Fish,Prochilodus nigricans,1,-68.736111,-12.693333,0.10,0.06,Total Mercury,NaN,NaN,2017-08-18 13:00:00,2017,2017
318,Martinez 2018,2017-08-18 13:00:00,Fish,Cynodon gibbus,1,-68.736111,-12.693333,0.72,0.41,Total Mercury,NaN,NaN,2017-08-18 13:00:00,2017,2017
319,Martinez 2018,2017-08-18 13:00:00,Fish,Pimelodus blochii,1,-68.736111,-12.693333,0.13,0.06,Total Mercury,NaN,NaN,2017-08-18 13:00:00,2017,2017
